In [1]:
import sys
import os
import logging
import pandas as pd
import datasets
from pprint import pprint
KEY = '1-CGMLSM'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

SPACE = {
    'DATA_RAW': f'_Data/0-Data_Raw',
    'DATA_RFT': f'_Data/1-Data_RFT',
    'DATA_CASE': f'_Data/2-Data_CASE',
    'DATA_CFDATA': f'_Data/3-Data_CFDATA',
    'DATA_SPLIT': f'_Data/4-Data_Split',
    'DATA_HFDATA': f'_Data/5-Data_HFData',
    'DATA_EXTERNAL': f'code/external',
    'CODE_FN': f'code/pipeline',
    'MODEL_ROOT': f'./_Model',
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'
print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

/home/jluo41/CGMLSM-Project/
code/pipeline


/home/jluo41/miniconda3/envs/nix/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from nn.cgmlsm.configuration_cgmlsm import CgmLsmConfig
from nn.cgmlsm.modeling_cgmlsm import CgmLsmLMHeadModel

In [3]:
HFDataName = 'OhioT1DM-Split_v0628'
path = os.path.join(SPACE['DATA_HFDATA'], HFDataName)
split_to_dataset = datasets.load_from_disk(path)
remove_unused_columns = True # if using the processed dataset, set to True. 
print(split_to_dataset)
Name_to_Data = {i: {'ds_tfm': split_to_dataset[i]} for i in split_to_dataset}
# exit()

DatasetDict({
    train: Dataset({
        features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
        num_rows: 44275
    })
    valid: Dataset({
        features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
        num_rows: 4925
    })
    test-id: Dataset({
        features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
        num_rows: 9214
    })
})


In [4]:
train_dataset  = Name_to_Data['train']['ds_tfm']
valid_dataset  = Name_to_Data['valid']['ds_tfm']
testid_dataset = Name_to_Data['test-id']['ds_tfm']

In [5]:
import sys
import os
import logging
import pandas as pd
import datasets
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    RobertaConfig,
    RobertaForSequenceClassification,
    PreTrainedTokenizerFast,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from pprint import pprint
from transformers import (
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
# Calculate AUC (Area Under the ROC Curve)
# For multi-class, we use one-vs-rest approach
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score

In [6]:
model_path = '_Model/CGMOnlyLSM/checkpoint__epoch2.16802__fingerprint'
model = CgmLsmLMHeadModel.from_pretrained(model_path)
model

CgmLsmLMHeadModel(
  (transformer): CgmLsmModel(
    (wte): Embedding(409, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x AttnBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=409, bias=False)
)

In [7]:
INPUT_LENGTH = 289
OUTPUT_LENGTH = 24

def collate_fn(batch):
    # pay attention that we have config in the function. 

    context_length = INPUT_LENGTH
    prediction_length = OUTPUT_LENGTH    
    # Extract and convert to tensors
    full_input_ids = [torch.tensor(ex['input_ids'] + ex['labels'], dtype=torch.long) for ex in batch]  # (B, 288)
    full_input_ids = torch.stack(full_input_ids)  # (B, 312)
    full_input_ids = full_input_ids[:, :context_length + prediction_length]

    results = {"input_ids": full_input_ids}

    # Create causal-style loss mask
    padded_labels = torch.full_like(full_input_ids, -100)
    padded_labels[:, context_length:context_length + prediction_length] = full_input_ids[:, context_length:context_length + prediction_length]  # only compute loss on the future
    results["labels"] = padded_labels
    
    return results

In [8]:
raw = train_dataset[:2]
batch = pd.DataFrame(raw).to_dict(orient="records")
# pprint(batch, compact=True)

batch = collate_fn(batch)
# batch = {k: v.to(device) for k, v in batch.items()}
batch = {k: v for k, v in batch.items()}
for k, v in batch.items():
    print(k, v.shape)

input_ids torch.Size([2, 313])
labels torch.Size([2, 313])


In [9]:
model.forward(**batch)

CausalLMOutputWithCrossAttentions(loss=tensor(2.4970, grad_fn=<NllLossBackward0>), logits=tensor([[[-0.8826, -5.0256, -4.9853,  ..., -4.8366, -4.2931, -4.2902],
         [-1.3482, -5.4329, -5.2097,  ..., -4.9946, -5.4601, -5.1279],
         [-0.2716, -5.8307, -5.5706,  ..., -5.5700, -5.7806, -5.4864],
         ...,
         [-2.6634, -8.5367, -8.4778,  ..., -8.7058, -8.8349, -8.6979],
         [-2.2007, -8.4539, -8.4988,  ..., -8.7192, -8.4551, -8.4097],
         [-2.6818, -8.5667, -8.7653,  ..., -8.6191, -8.5666, -8.3971]],

        [[-0.0451, -5.5894, -5.5561,  ..., -5.1793, -4.8048, -4.8324],
         [-0.4077, -6.3586, -6.1120,  ..., -6.2715, -6.4057, -6.1858],
         [-0.8342, -6.4335, -6.1002,  ..., -6.3820, -6.5699, -6.1319],
         ...,
         [-2.1531, -8.4803, -8.5012,  ..., -8.7239, -8.4763, -8.3895],
         [-2.6855, -8.8327, -8.9613,  ..., -8.8431, -8.7338, -8.6721],
         [-2.5599, -7.5403, -7.9881,  ..., -7.9071, -8.0886, -7.4993]]],
       grad_fn=<UnsafeView

# Inference

In [10]:


def collate_fn_predict(batch):
    context_length = INPUT_LENGTH
    prediction_length = OUTPUT_LENGTH    
    # Extract and convert to tensors
    full_input_ids = [torch.tensor(ex['input_ids'], dtype=torch.long) for ex in batch]  # (B, 288)
    full_input_ids = torch.stack(full_input_ids)  # (B, 312)
    full_input_ids = full_input_ids[:, :context_length]

    # real_series = [ex['input_ids'] for ex in batch]
    
    labels = [torch.tensor(ex['labels'], dtype=torch.long) for ex in batch]
    labels = torch.stack(labels)
    results = {
        "input_ids": full_input_ids,
        "labels": labels,
    }

    results['stratum'] = ['None' for ex in batch]
    results['split_timebin'] = ['None' for ex in batch]
   
    return results

In [11]:
raw = train_dataset[:16]
batch = pd.DataFrame(raw).to_dict(orient="records")
# pprint(batch, compact=True)
batch

[{'PID': 110000000002,
  'final_split': 'train',
  'input_ids': [76,
   72,
   68,
   65,
   63,
   66,
   71,
   78,
   90,
   99,
   110,
   121,
   131,
   137,
   140,
   140,
   143,
   142,
   136,
   125,
   128,
   127,
   128,
   129,
   129,
   127,
   124,
   123,
   124,
   126,
   123,
   123,
   124,
   125,
   128,
   126,
   124,
   122,
   123,
   117,
   108,
   104,
   100,
   98,
   95,
   90,
   85,
   81,
   77,
   73,
   72,
   71,
   66,
   60,
   54,
   50,
   48,
   49,
   54,
   60,
   71,
   72,
   74,
   77,
   84,
   100,
   118,
   134,
   143,
   139,
   139,
   133,
   126,
   118,
   109,
   102,
   96,
   93,
   93,
   93,
   94,
   93,
   92,
   93,
   95,
   92,
   89,
   90,
   92,
   94,
   94,
   92,
   94,
   96,
   98,
   97,
   97,
   98,
   97,
   94,
   89,
   86,
   81,
   77,
   74,
   73,
   71,
   69,
   68,
   70,
   74,
   81,
   89,
   95,
   103,
   113,
   120,
   126,
   132,
   137,
   137,
   136,
   133,
   133,
   133,
   132,


In [12]:

batch = collate_fn_predict(batch)
# batch = {k: v.to(device) for k, v in batch.items() if k not in ['stratum', 'split_timebin']}
batch = {k: v for k, v in batch.items() if k not in ['stratum', 'split_timebin']}

for k, v in batch.items():
    print(k, v.shape)

input_ids torch.Size([16, 289])
labels torch.Size([16, 24])


In [13]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    max_new_tokens=OUTPUT_LENGTH, # 24, # 96, # config.prediction_length,
    do_sample=False,             # use sampling instead of greedy decoding
    pad_token_id=model.config.pad_token_id,  # needed for GPT2-style models
    eos_token_id=model.config.eos_token_id,
    bos_token_id=model.config.bos_token_id,
    use_cache=True,
)

print(generation_config)

GenerationConfig {
  "bos_token_id": 407,
  "eos_token_id": 408,
  "max_new_tokens": 24,
  "pad_token_id": 0
}



In [14]:
import numpy as np

def calculate_rmse_at_horizons(real, pred, horizons=[6, 12, 24]):
    """
    Calculate RMSE at different prediction horizons.
    
    Args:
        real (list): List of real values
        pred (list): List of predicted values
        horizons (list): List of horizons to calculate RMSE for
        
    Returns:
        dict: Dictionary containing RMSE values for each horizon
    """
    results = {}
    
    for horizon in horizons:
        # Calculate squared differences for each sequence
        squared_diffs = []
        
        # Convert to numpy arrays and take first horizon elements
        r_horizon = np.array(real)[:horizon]
        p_horizon = np.array(pred)[:horizon]
        
        # Calculate squared differences
        diff = r_horizon - p_horizon
        squared_diff = np.mean(diff ** 2)
        squared_diffs.append(squared_diff)
    
        # Calculate RMSE across all sequences
        rmse = np.sqrt(np.mean(squared_diffs))
        results[f'RMSE_{horizon}'] = rmse
        
    return results

# Calculate RMSE for the validation set

In [15]:
outputs = model.generate(**{k: v for k, v in batch.items() if k not in ['stratum', 'split_timebin']}, generation_config=generation_config)
# outputs.shape

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

model.to(device)

CgmLsmLMHeadModel(
  (transformer): CgmLsmModel(
    (wte): Embedding(409, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x AttnBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=409, bias=False)
)

In [17]:
def get_prediction_results(model, dataset, batch_size, generation_config):


    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,  # Keep validation data in order
        collate_fn=collate_fn_predict  # Use the same collate function as training
    )

    print(len(dataloader))

    # Iterate through batches

    all_inputs = [] 
    all_reals = []
    all_preds = []
    all_stratum = []
    all_split_timebin = []
    for batch_idx, batch in enumerate(dataloader):
        # Process each batch
        # print(f"Batch {batch_idx}:")
        # print(f"Input shape: {batch['input_ids'].shape}")
        # print(f"Labels shape: {batch['labels'].shape}")
        # Add your processing logic here

        batch_inputs = {k: v.to(device) for k, v in batch.items() if k not in ['stratum', 'split_timebin']}
        # for k, v in batch.items():
        #     print(k, v.shape)
        # outputs = model.generate(**batch, generation_config=generation_config)
        outputs = model.generate(**batch_inputs, generation_config=generation_config)
        # print(outputs.shape)

        pred = outputs[:, -generation_config.max_new_tokens: ]
        # print(pred.shape)


        inputs = batch['input_ids'].cpu().numpy() 
        real = batch['labels'].cpu().numpy()
        pred = pred.cpu().numpy()
        # print(pred.shape)

        for i in range(len(pred)):
            pred_dp = pred[i].tolist()
            real_dp = real[i].tolist()
            input_dp = inputs[i].tolist()
            if np.std(pred_dp) == 0:
                pred_dp = [input_dp[-1]] * len(pred_dp)
            all_inputs.append(input_dp)
            all_reals.append(real_dp)
            all_preds.append(pred_dp)
            all_stratum.append(batch['stratum'][i])
            all_split_timebin.append(batch['split_timebin'][i])

        if batch_idx % 5 == 0:
            print('-------------------', batch_idx, datetime.now())


    df_pred_results = pd.DataFrame({'stratum': all_stratum, 'split_timebin': all_split_timebin, 'input': all_inputs, 'real': all_reals, 'pred': all_preds, })
    return df_pred_results

In [18]:
# Create a DataLoader for the validation dataset
from torch.utils.data import DataLoader
from datetime import datetime 
batch_size = 32
dataset = testid_dataset

df_pred_results = get_prediction_results(model, dataset, batch_size, generation_config)

288
------------------- 0 2025-07-09 23:48:54.593930
------------------- 5 2025-07-09 23:48:55.719765
------------------- 10 2025-07-09 23:48:56.845871
------------------- 15 2025-07-09 23:48:57.970531
------------------- 20 2025-07-09 23:48:59.094497
------------------- 25 2025-07-09 23:49:00.218163
------------------- 30 2025-07-09 23:49:01.340457
------------------- 35 2025-07-09 23:49:02.462968
------------------- 40 2025-07-09 23:49:03.586753
------------------- 45 2025-07-09 23:49:04.710412
------------------- 50 2025-07-09 23:49:05.833416
------------------- 55 2025-07-09 23:49:06.958433
------------------- 60 2025-07-09 23:49:08.081165
------------------- 65 2025-07-09 23:49:09.203818
------------------- 70 2025-07-09 23:49:10.327146
------------------- 75 2025-07-09 23:49:11.450067
------------------- 80 2025-07-09 23:49:12.572194
------------------- 85 2025-07-09 23:49:13.694279
------------------- 90 2025-07-09 23:49:14.816313
------------------- 95 2025-07-09 23:49:15.93904

In [19]:
df = df_pred_results

row = df.iloc[0]
real = row['real']
pred = row['pred']
rmse_results = calculate_rmse_at_horizons(real, pred)
print("\nRMSE Results:")
for horizon, value in rmse_results.items():
    print(f"{horizon}: {value:.4f}")



RMSE Results:
RMSE_6: 4.5092
RMSE_12: 30.9731
RMSE_24: 69.9544


In [20]:
results = df.apply(lambda x: calculate_rmse_at_horizons(x['real'], x['pred']), axis = 1)
results

0       {'RMSE_6': 4.509249752822894, 'RMSE_12': 30.97...
1       {'RMSE_6': 8.972179222463181, 'RMSE_12': 36.98...
2       {'RMSE_6': 14.53157481715821, 'RMSE_12': 43.90...
3       {'RMSE_6': 20.960279896349984, 'RMSE_12': 50.4...
4       {'RMSE_6': 28.142494558940577, 'RMSE_12': 56.7...
                              ...                        
9209    {'RMSE_6': 2.6457513110645907, 'RMSE_12': 3.29...
9210    {'RMSE_6': 8.031189202104505, 'RMSE_12': 7.762...
9211    {'RMSE_6': 3.6742346141747673, 'RMSE_12': 4.80...
9212    {'RMSE_6': 1.4142135623730951, 'RMSE_12': 7.97...
9213    {'RMSE_6': 1.9578900207451218, 'RMSE_12': 9.58...
Length: 9214, dtype: object

In [22]:
pd.DataFrame(results.to_list()).describe()

,RMSE_6,RMSE_12,RMSE_24
count,9214.000000,9214.000000,9214.000000
mean,9.024068,15.894809,26.876169
std,8.224815,13.882210,21.052455
min,0.000000,0.500000,1.099242
25%,3.341656,6.070763,11.550613
50%,6.467869,11.726039,21.169947
75%,11.979149,21.154688,36.059846
max,78.853239,108.628188,171.451038


In [26]:
model_path = '_Model/glucopred_nix_optuna/CGMOnlyLSM'

if not os.path.exists(model_path):
    os.makedirs(model_path)

In [ ]:
df_pred_results = df_pred_results.rename(columns={'input': 'Inputs', 'real': 'Real', 'pred': 'Predict'})
df_pred_results['Splitname'] = 'test-id'
df_pred_results

In [51]:
import os
path = os.path.join(model_path, 'df_predictions.parquet')
df_pred_results.to_parquet(path)

In [52]:

def evaluate_predictions(df_predict, subgroup_config_list, horizon_to_se, metric_list):
    """Evaluate predictions using SeqPredEval"""
   
    df_predict['prediction_hour'] = None

    try:
        df_predict['age_interval'] = df_predict['stratum'].apply(lambda x: x.split('_')[0])
        df_predict['gender'] = df_predict['stratum'].apply(lambda x: x.split('_')[1])
        df_predict['diabetes_type'] = df_predict['stratum'].apply(lambda x: x.split('_')[2])
    except:
        df_predict['age_interval'] = 'unknown'
        df_predict['gender'] = 'unknown'
        df_predict['diabetes_type'] = 'T1DM'

    # Configure evaluation parameters
    x_hist_seq_name = 'Inputs'
    y_real_seq_name = 'Real'
    y_pred_seq_name = 'Predict'
    
    # Filter and prepare data for evaluation
    df_predict_to_eval = df_predict
    df_case_eval = df_predict_to_eval

    # Create evaluation instance and run evaluation
    eval_instance = SeqPredEval(
        df_case_eval=df_case_eval,
        subgroup_config_list=subgroup_config_list,
        x_hist_seq_name=x_hist_seq_name,
        y_real_seq_name=y_real_seq_name,
        y_pred_seq_name=y_pred_seq_name,
        metric_list=metric_list,
        horizon_to_se=horizon_to_se,
    )

    # Get evaluation results
    df_report_neat = eval_instance.df_report_neat.copy()
    df_report_full = eval_instance.df_report_full.copy()

    return df_report_neat, df_report_full

In [54]:
from nn.eval.seqeval import SeqPredEval


df_predict = df_pred_results


metric_list = ['rMSE'] 

subgroup_config_list = [
    ['Splitname'],
]

horizon_to_se = {
    '000-030min': [0, 6],
    '000-060min': [0, 12],
    '000-120min': [0, 24],
}

df_report_neat, df_report_full = evaluate_predictions(df_predict, subgroup_config_list, horizon_to_se, metric_list)
df_report_neat

['rMSE']


,setname,000-030min_rMSE,000-060min_rMSE,000-120min_rMSE
0,"(test-id,)",9.024059,15.894782,26.876165


In [55]:
import os

path = os.path.join(model_path, 'eval.parquet')
df_report_full.to_parquet(path)